## 4-1. 코랩 노트북
- 모델을 파인튜닝하려면 하드웨어 가속기를 사용해야 계산 속도를 높일 수 있습니다.
- 코랩에서는 GPU와 TPU 두 종류의 가속기를 지원합니다.
- 코랩 화면의 연결 GPU 메뉴 탭에서 런타임 > 런타임 유형 변경을 클릭합니다.
- 이후 GPU 혹은 TPU 둘 중 하나를 선택합니다.

* 파인튜닝(fine-tuning)은 기계 학습 모델을 특정 작업이나 데이터에 더 잘 맞도록 조정하는 과정을 의미.
주로 미리 학습된(pre-trained) 모델을 가져와서 새로운 작업에 맞게 조정하는 방식으로 사용

## 4-2. 각종 설정하기
### 1. TPU 관련 패키지를 설치 -> GPU 사용 (할 필요 없음)
TPU 관련 패키지를 설치합니다. TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요. GPU를 쓴다면 아래 라인을 실행할 필요가 없습니다.

In [ ]:
# 코드 4-1 TPU 관련 패키지 설치
# TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요.
# pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

####2. 의존성 패키지 설치
TPU 이외에 의존성 있는 패키지를 설치합니다.

In [1]:
# 코드 4-2 의존성 패키지 설치
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


####3. 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결
자신의 구글 계정에 적용됩니다.

In [2]:
# 코드 4-3 구글드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


####4. 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.
- PyTorch 사용
- kcbert-base 모델을 NSMC 데이터로 파인튜닝을 해보겠습니다.

함수 ClassificationTrainArguments의 각 argument(인자)가 하는 역할
- pretrained_model_name : 프리트레인 마친 언어모델의 이름(단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)
- downstream_corpus_name : 다운스트림 데이터의 이름.
- downstream_model_dir : 파인튜닝된 모델의 체크포인트가 저장될 위치. /gdrive/My Drive/nlpbook/checkpoint-doccls라고 함은 자신의 구글 드라이브의 내 폴더 하위의 nlpbook/checkpoint-doccls 디렉토리에 모델 체크포인트가 저장됩니다.
- batch_size : 배치 크기. 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available() == True)했다면 32, TPU라면(torch.cuda.is_available() == False) 4. 코랩 환경에서 TPU는 보통 8개 코어가 할당되는데 batch_size는 코어별로 적용되는 배치 크기이기 때문에 이렇게 설정해 둡니다.
- learning_rate : 러닝레이트. 1회 스텝에서 한 번에 얼마나 업데이트할지에 관한 크기를 가리킵니다. 이와 관련한 자세한 내용은 3-2-2장 Technics를 참고하세요.
- max_seq_length : 토큰 기준 입력 문장 최대 길이. 이보다 긴 문장은 max_seq_length로 자르고, 짧은 문장은 max_seq_length가 되도록 스페셜 토큰(PAD)을 붙여 줍니다.
- epochs : 학습 에폭 수. 3이라면 학습 데이터를 3회 반복 학습합니다.
- tpu_cores : TPU 코어 수. 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available() == True)했다면 0, TPU라면(torch.cuda.is_available() == False) 8.
- seed : 랜덤 시드(정수, integer). None을 입력하면 랜덤 시드를 고정하지 않습니다.

In [3]:
# 코드 4-4 모델 환경 설정
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base", # 프리트레인 마친 언어모델의 이름
    downstream_corpus_name="nsmc", # 다운스트림 데이터의 이름.
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls", # 파인튜닝된 모델의 체크포인트가 저장될 위치
    batch_size=32 if torch.cuda.is_available() else 4, # batch_size
    learning_rate=5e-5, # learning rate
    max_seq_length=128, # 토큰 기준 입력 문장 최대 길이
    epochs=3, # 학습 횟수
    tpu_cores=0 if torch.cuda.is_available() else 8, # TPU 코어 수
    seed=7, # 랜덤 시드
)

####5. 랜덤 시드 고정
- 학습 재현을 위해 랜덤 시드를 고정합니다.
- args에 지정된 시드로 고정하는 역할

Tip: 난수는 배치(batch)를 뽑거나 드롭아웃 대상 뉴런의 위치를 정할 때 등 다양하게 쓰입니다.
컴퓨터는 난수 생성 알고리즘을 사용해 난수를 만들어내는데요. 이 때 난수 생성 알고리즘을 실행하기 위해 쓰는 수를 랜덤 시드라고 합니다.
만일 같은 시드를 사용한다면 컴퓨터는 계속 같은 패턴의 난수를 생성하게 됩니다.

In [4]:
# 코드 4-5 랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


####6. 로거 설정
- 메세지 출력 등을 위한 logger를 설정합니다.
- 각종 로그들을 출력하는 로거를 설정

In [5]:
# 코드 4-6 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_siz

###4-3. 말뭉치 다운로드
- 실습에 사용할 말뭉치(Naver Sentiment Movie review Corpus)를 다운로드합니다.
- 데이터를 내려받는 도구로 코포라(Korpora)라는 오픈소스 파이썬 패키지를 사용해, corpus_name(nsmc)에 해당하는 말뭉치를 코랩 환경 로컬의 root_dir(/content/Korpora) 이하에 저장해 둡니다.

In [6]:
# 코드 4-7 말뭉치 다운로드
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 116MB/s]                             
[nsmc] download ratings_test.txt: 4.90MB [00:00, 41.8MB/s]                            


### 4-4. 토크나이저 준비
- 토큰화를 수행하는 토크나이저를 선언합니다.
- 이, 코드에서 다루는 데이터의 기본 단위는 텍스트 형태의 문장(sentence)입니다.
- 토큰화(tokenization)란 문장을 토큰(token) 시퀀스로 분절하는 과정을 가리킵니다.
- 여기서 사용하는 모델은 자연어 문장을 분절한 토큰 시퀀스를 입력으로 받습니다.
- 아래의 코드를 실행해서 kcbert-base 모델이 사용하는 토크나이저(tokenizer)를 선언합니다.
- 토크나이저(tokenizer)는 토큰화를 수행하는 프로그램이라는 뜻입니다.

In [7]:
# 코드 4-8 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

### 4-5. 데이터 전처리 하기
- 딥러닝 모델을 학습하려면 학습데이터를 배치(batch) 단위로 지속적으로 모델에 공급해 주어야 합니다.
- 파이토치(PyTorch)에서는 이 역할을 데이터 로더(DataLoader)가 수행합니다. 이건 TF(Tensorflow)도 동일합니다.

- 아래의 코드를 실행하면 데이터셋(Dataset)을 만들수 있습니다.
- NsmcCorpus는 CSV 파일 형식의 NSMC 데이터를 “문장(영화 리뷰) + 레이블(긍정, 부정)” 형태로 읽어들이는 역할을 합니다.
- ClassificationDataset 함수는 그림 데이터셋의 역할을 수행합니다.



In [9]:
# 코드 4-9 학습 데이터셋 구축
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_BertTokenizer_128_nsmc_document-classification [took 6.734 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_BertTokenizer_128_nsmc_document-classification [took 6.734 s]


- 이 클래스는 NsmcCorpus와 토크나이저 파트에서 선언해 둔 토크나이저를 품고 있는데요.
- NsmcCorpus가 넘겨준 데이터(문장, 레이블)를 모델이 학습할 수 있는 형태로 가공합니다.
- 다시 말해 문장을 토큰화하고 이를 인덱스로 변환하는 한편, 레이블 역시 정수(integer)로 바꿔주는 역할을 합니다.

#### 학습 데이터 로더 구축
- 학습 중에 평가할 테스트 데이터를 구축합니다.
- ClassificationDataset 클래스는 학습데이터에 속한 각각의 문장을 input_ids, attention_mask, token_type_ids, label 등 네 가지로 변환한 형태로 가지고 있습니다.
- 이터 로더는 ClassificationDataset 클래스가 들고 있는 전체 인스턴스 가운데 배치 크기(이미 정의한 args의 batch_size)만큼을 뽑아 배치 형태로 가공하는 역할을 수행합니다.

In [10]:
# 코드 4-10 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False), # 데이터 로더로 배치 만들때 batch_size 개수만큼 비복원 랜덤 추출
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

- 코드를 실행하면 평가용 데이터 로더를 구축할 수 있습니다.
- 학습용 데이터 로더와 달리 평가용 데이터 로더는 SequentialSampler를 사용하고 있음을 알 수 있습니다.
```
# SequentialSampler는 batch_size만큼의 갯수만큼을 인스턴스 순서대로 추출하는 역할을 합니다.
```
- 학습때 Batch 구성은 랜덤으로 하는것이 좋음 -> 평가할 때 평가(Test)용 데이터셋 전체를 사용하는데 굳이? 랜덤으로 구성할 이유가 없음.


In [11]:
# 코드 4-11 평가용 데이터 로더 구축
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 14.851 s]
INFO:ratsnlp:tokenize sentences [took 14.851 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

### 4-6. 모델 초기화
- 프리트레인이 완료된 BERT 모델을 읽고, 문서 분류를 수행할 모델을 초기화합니다.
- BertForSequenceClassification 은 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스입니다.

```
BertForSequenceClassification - huggingface에서 제공하는 transformer 라이브러리에 있음
```



In [12]:
# 코드 4-12 모델 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [13]:
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly i

### 4-7. 모델 학습 시키기
- 파이토치 라이트닝(pytorch lightning)이 제공하는 라이트닝 모듈(LightningModule) 클래스를 상속받아 태스크(task)를 정의합니다.
- Task에는 모델과 옵티마이저(optimizer), 학습 과정(training process) 등이 정의돼 있습니다.
- 모델은 위의 코드에서 준비한 모델 클래스를 ClassificationTask에 포함시킵니다.
- ClassificationTask 클래스에는 옵티마이저, 러닝레이트 스케줄러(learning rate scheduler)가 정의되어 있는데요.
- 옵티마이저로는 아담(Adam), 러닝레이트 스케줄러로는 ExponentialLR을 사용합니다.

In [14]:
# 코드 4-13 TASK 정의
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

- 트레이너는 파이토치 라이트닝 라이브러리의 도움을 받아 GPU/TPU 설정, 로그 및 체크포인트 등 귀찮은 설정들을 알아서 해줍니다.

In [15]:
# 코드 4-14 TRAINER 정의
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


- 준비한 데이터와 모델로 학습을 시작합니다.
- 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-doccls`)에 저장됩니다.
- 트레이너의 fit 함수를 호출하면 학습이 시작됩니다

In [16]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
